In [1]:
import numpy as np
import rasterio
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import os
from netCDF4 import Dataset as NetCDFFile 
import h5netcdf
from mpl_toolkits.axes_grid1 import make_axes_locatable
import rioxarray as rio
import geopandas as gpd
import seaborn as sns
from mpl_toolkits.basemap import Basemap

In [2]:
#load in temp data
# temp_1999 = xr.open_dataset("../Temperature/Output Deliverable/export_1999.nc")
# temp_2008 = xr.open_dataset("../Temperature/Output Deliverable/export_2008.nc")
# temp_change = xr.open_dataset("../Temperature/Output Deliverable/export_change.nc")
#load in netcdf file
change = NetCDFFile('../Temperature/Output Deliverable/export_change.nc')


#load in whale data
loc_1999 = pd.read_csv('exports/loc_1999.csv').drop('geometry', axis=1)
loc_2008 = pd.read_csv('exports/loc_2008.csv').drop('geometry', axis=1)
#turn to gpd
loc_1999 = gpd.GeoDataFrame(loc_1999, geometry=gpd.points_from_xy(x=loc_1999['long'], y=loc_1999['lat']))
loc_2008 = gpd.GeoDataFrame(loc_2008, geometry=gpd.points_from_xy(x=loc_2008['long'], y=loc_2008['lat']))

OSError: [Errno -51] NetCDF: Unknown file format: '../Temperature/Output Deliverable/export_change.nc'

In [ ]:
#read the variable in the netcdf file:
lat = change.variables['lat'][:] # need it to be from -180 to 180 
long = change.variables['lon'][:]
sst = change.variables['change_percent'][:] #percent change in temerature from 1999 - 2008

In [ ]:
# Define the corners of the map
llon = -165.662
llat = 4.606
ulon = -54.362
ulat = 53.148

# Create a Basemap object
map = Basemap(projection='merc', llcrnrlon=llon, llcrnrlat=llat, urcrnrlon=ulon, urcrnrlat=ulat, resolution='f')

# Draw map features
map.drawcoastlines()
map.drawcountries()
map.drawlsmask(land_color='Linen', ocean_color='#CCFFFF')

# Generate some sample data (replace this with your actual data)
lons = long - 360
lats = lat
lons, lats = np.meshgrid(lons, lats)
sst = sst  # Sample sea surface temperature data

# Transform coordinates for plotting on the map
x, y = map(lons, lats)

# Plot the contour plot on the map
temp = map.contourf(x, y, sst)

# Generate some sample data points to overlay on the map
data_lons = loc_2008.long
data_lats = loc_2008.lat

# Transform coordinates for plotting data points on the map
data_x, data_y = map(data_lons, data_lats)

# Plot the data points on the map
map.plot(data_x, data_y, 'or', markersize=1)  # blue circles with size 5

# Add a colorbar
cb = map.colorbar(temp, location='bottom', size="5%", pad="2%")
cb.set_label('Percent Change')

plt.clabel(temp, fontsize=9, inline=1) # contour labels
plt.title('Mean Sea Level Pressure')
# Show the plot
plt.show()
plt.savefig('temp.png')


In [ ]:
sst